# Customer.IO Data Pipelines API - Device Management

## Purpose

This notebook demonstrates comprehensive device registration and management with Customer.IO's Data Pipelines API.
It covers mobile device registration, push notification setup, device attribute management, and cross-platform device tracking with proper validation and error handling.

## Prerequisites

- Complete setup from `00_setup_and_configuration.ipynb`
- Complete authentication setup from `01_authentication_and_utilities.ipynb`
- Customer.IO API key configured in Databricks secrets
- Understanding of mobile push notification concepts

## Key Concepts

- **Device Registration**: Linking devices to users for push notifications
- **Push Tokens**: Device-specific identifiers for notification delivery
- **Platform Types**: iOS, Android, and web push notification differences
- **Device Attributes**: Custom properties for targeting and segmentation
- **Token Management**: Registration, updates, and cleanup
- **Cross-Platform Tracking**: Users with multiple devices

## Device Types Covered

1. **iOS Devices**: APNS (Apple Push Notification Service) integration
2. **Android Devices**: FCM (Firebase Cloud Messaging) integration
3. **Web Browsers**: Web Push API for browser notifications
4. **Desktop Applications**: Cross-platform desktop push notifications
5. **Smart Devices**: IoT and embedded device tracking

## Setup and Imports

In [ ]:
# Standard library imports
import sys
import os
from datetime import datetime, timezone, timedelta
from typing import Dict, List, Optional, Any, Union, Set
import json
import uuid
from enum import Enum
import re

print("SUCCESS: Standard libraries imported")

In [ ]:
# Add utils directory to Python path
sys.path.append('/Workspace/Repos/customer_io_notebooks/utils')
print("SUCCESS: Utils directory added to Python path")

In [ ]:
# Import Customer.IO API utilities
from utils.api_client import CustomerIOClient
from utils.validators import (
    DeviceRequest,
    validate_request_size,
    create_context
)

print("SUCCESS: Customer.IO API utilities imported")

In [ ]:
# Import transformation utilities
from utils.transformers import (
    BatchTransformer,
    ContextTransformer
)

print("SUCCESS: Transformation utilities imported")

In [ ]:
# Import error handling utilities
from utils.error_handlers import (
    CustomerIOError,
    RateLimitError,
    ValidationError,
    NetworkError,
    retry_on_error,
    ErrorContext
)

print("SUCCESS: Error handling utilities imported")

In [ ]:
# Import Databricks and Spark utilities
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable

print("SUCCESS: Databricks and Spark utilities imported")

In [ ]:
# Import validation and logging and DeviceManager
import structlog
from pydantic import ValidationError as PydanticValidationError, BaseModel, Field, validator

# Import DeviceManager for advanced device management
from utils.device_manager import (
    DeviceManager,
    DeviceType,
    PushProvider, 
    DeviceStatus,
    DeviceInfo,
    DeviceAttributes,
    DeviceRegistration
)

# Initialize logger
logger = structlog.get_logger("device_management")

print("SUCCESS: Validation, logging, and DeviceManager imported")

## Configuration and Client Setup

In [ ]:
# Load configuration from setup notebook (secure approach)
try:
    CUSTOMERIO_REGION = dbutils.widgets.get("customerio_region") or "us"
    DATABASE_NAME = dbutils.widgets.get("database_name") or "customerio_demo"
    CATALOG_NAME = dbutils.widgets.get("catalog_name") or "main"
    ENVIRONMENT = dbutils.widgets.get("environment") or "test"
    
    print(f"Configuration loaded from setup notebook:")
    print(f"  Region: {CUSTOMERIO_REGION}")
    print(f"  Database: {CATALOG_NAME}.{DATABASE_NAME}")
    print(f"  Environment: {ENVIRONMENT}")
    
except Exception as e:
    print(f"WARNING: Could not load configuration from setup notebook: {str(e)}")
    print("INFO: Using fallback configuration")
    CUSTOMERIO_REGION = "us"
    DATABASE_NAME = "customerio_demo"
    CATALOG_NAME = "main"
    ENVIRONMENT = "test"

In [ ]:
# Get Customer.IO API key from secure storage
CUSTOMERIO_API_KEY = dbutils.secrets.get("customerio", "api_key")
print("SUCCESS: Customer.IO API key retrieved from secure storage")

In [ ]:
# Configure Spark to use the specified database
spark.sql(f"USE {CATALOG_NAME}.{DATABASE_NAME}")
print("SUCCESS: Database configured")

In [ ]:
# Initialize the Customer.IO client and DeviceManager
try:
    client = CustomerIOClient(
        api_key=CUSTOMERIO_API_KEY,
        region=CUSTOMERIO_REGION,
        timeout=30,
        max_retries=3,
        retry_backoff_factor=2.0,
        enable_logging=True,
        spark_session=spark
    )
    print("SUCCESS: Customer.IO client initialized for device management")
    
    # Initialize DeviceManager
    device_manager = DeviceManager(client)
    print("SUCCESS: DeviceManager initialized")
    
except Exception as e:
    print(f"ERROR: Failed to initialize Customer.IO client: {str(e)}")
    raise

## Test-Driven Development: Device Validation Functions

In [ ]:
# Test device registration validation using DeviceManager models

def test_device_registration_validation():
    """Test that device registrations have required fields and pass validation."""
    
    try:
        # Test valid iOS device using DeviceManager models
        device_info = DeviceInfo(
            token="abc123def456ghi789",
            type=DeviceType.IOS,
            device_id="iPhone14,2",
            os_version="17.2",
            app_version="2.1.0"
        )
        
        device_attributes = DeviceAttributes(
            push_enabled=True,
            timezone="America/New_York",
            locale="en-US"
        )
        
        device_registration = DeviceRegistration(
            user_id="test_user_001",
            device_info=device_info,
            attributes=device_attributes
        )
        
        assert device_registration.device_info.type == "ios"
        assert device_registration.device_info.token == "abc123def456ghi789"
        assert device_registration.user_id == "test_user_001"
        print("SUCCESS: iOS device validation test passed")
        return True
    except Exception as e:
        print(f"ERROR: iOS device validation test failed: {str(e)}")
        return False

# Run the test
test_device_registration_validation()

In [ ]:
# Test push token validation using DeviceManager models

def test_push_token_validation():
    """Test that push tokens are validated for different platforms."""
    
    try:
        # Test iOS token format (64 hex characters)
        ios_token = "a1b2c3d4e5f6789012345678901234567890123456789012345678901234abcd"
        ios_device = DeviceInfo(
            token=ios_token,
            type=DeviceType.IOS,
            push_provider=PushProvider.APNS
        )
        assert len(ios_device.token) == 64
        
        # Test Android token format (FCM token)
        android_token = "fGcm_token_123:APA91bH...example_token"
        android_device = DeviceInfo(
            token=android_token,
            type=DeviceType.ANDROID,
            push_provider=PushProvider.FCM
        )
        assert len(android_device.token) >= 10
        
        # Test web push token format
        web_token = "BP91bH_example_web_push_token_with_base64_encoding"
        web_device = DeviceInfo(
            token=web_token,
            type=DeviceType.WEB,
            push_provider=PushProvider.WEB_PUSH
        )
        assert len(web_device.token) >= 10
        
        print("SUCCESS: Push token validation tests passed")
        return True
    except Exception as e:
        print(f"ERROR: Push token validation test failed: {str(e)}")
        return False

# Run the test
test_push_token_validation()

In [ ]:
# Test device attributes validation using DeviceManager models

def test_device_attributes_validation():
    """Test that device attributes are properly structured and typed."""
    
    try:
        device_attributes = DeviceAttributes(
            push_enabled=True,
            timezone="America/New_York",
            locale="en-US",
            battery_level=0.85,
            network_type="wifi",
            screen_width=1179,
            screen_height=2556
        )
        
        # Validate data types and constraints
        assert isinstance(device_attributes.push_enabled, bool)
        assert 0.0 <= device_attributes.battery_level <= 1.0
        assert device_attributes.timezone == "America/New_York"
        assert device_attributes.locale == "en-US"
        assert device_attributes.screen_width > 0
        assert device_attributes.screen_height > 0
        
        print("SUCCESS: Device attributes validation test passed")
        return True
    except Exception as e:
        print(f"ERROR: Device attributes validation test failed: {str(e)}")
        return False

# Run the test
test_device_attributes_validation()

## Device Types and Platform Support

In [ ]:
# Test device types and platform support from DeviceManager

print("Available device types from DeviceManager:")
for device_type in DeviceType:
    print(f"  - {device_type.value}")

print("\nAvailable push providers from DeviceManager:")
for provider in PushProvider:
    print(f"  - {provider.value}")

print("\nAvailable device statuses from DeviceManager:")
for status in DeviceStatus:
    print(f"  - {status.value}")

print("SUCCESS: Device types and enumerations are available from DeviceManager")

## Type-Safe Device Models

In [ ]:
# Test DeviceInfo model from DeviceManager

ios_device_info = DeviceInfo(
    token="a1b2c3d4e5f6789012345678901234567890123456789012345678901234abcd",
    type=DeviceType.IOS,
    device_id="iPhone14,2",
    device_model="iPhone 15 Pro",
    os_version="17.2",
    app_version="2.1.0",
    push_provider=PushProvider.APNS
)

print("DeviceInfo model test:")
print(f"  Token: {ios_device_info.token[:20]}...")  # Truncate for display
print(f"  Type: {ios_device_info.type}")
print(f"  Device ID: {ios_device_info.device_id}")
print(f"  Model: {ios_device_info.device_model}")
print(f"  OS Version: {ios_device_info.os_version}")
print(f"  Push Provider: {ios_device_info.push_provider}")
print("SUCCESS: DeviceInfo model working with validation")

In [ ]:
# Test DeviceAttributes model from DeviceManager

ios_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/New_York",
    locale="en-US",
    battery_level=0.85,
    network_type="wifi",
    screen_width=1179,
    screen_height=2556
)

print("DeviceAttributes model test:")
print(f"  Push Enabled: {ios_attributes.push_enabled}")
print(f"  Timezone: {ios_attributes.timezone}")
print(f"  Locale: {ios_attributes.locale}")
print(f"  Battery Level: {ios_attributes.battery_level}")
print(f"  Network Type: {ios_attributes.network_type}")
print(f"  Screen Size: {ios_attributes.screen_width}x{ios_attributes.screen_height}")
print("SUCCESS: DeviceAttributes model working with validation")

In [ ]:
# Test DeviceRegistration model from DeviceManager

ios_registration = DeviceRegistration(
    user_id="user_mobile_001",
    device_info=ios_device_info,
    attributes=ios_attributes,
    status=DeviceStatus.ACTIVE
)

print("DeviceRegistration model test:")
print(f"  User ID: {ios_registration.user_id}")
print(f"  Device Type: {ios_registration.device_info.type}")
print(f"  Device Model: {ios_registration.device_info.device_model}")
print(f"  Status: {ios_registration.status}")
print(f"  Push Enabled: {ios_registration.attributes.push_enabled}")
print(f"  Registered At: {ios_registration.registered_at}")
print("SUCCESS: DeviceRegistration model working with validation")

## Device Registration Implementation

In [ ]:
# Test device registration using DeviceManager

print("iOS device registration using DeviceManager:")
print(f"  User: {ios_registration.user_id}")
print(f"  Device: {ios_registration.device_info.device_model}")
print(f"  Type: {ios_registration.device_info.type}")
print(f"  Push Provider: {ios_registration.device_info.push_provider}")
print(f"  Token: {ios_registration.device_info.token[:20]}...")
print(f"  Status: {ios_registration.status}")
print("SUCCESS: Device registration model created with DeviceManager")

In [ ]:
# Register device using DeviceManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - device not actually registered")
    result = {"status": "test_success", "message": "Device registration validated"}
else:
    result = device_manager.register_device(ios_registration)

print(f"Device registration result: {result}")
print("SUCCESS: Device registered using DeviceManager")

## Multi-Platform Device Support

In [ ]:
# Create Android device registration using DeviceManager models

android_device_info = DeviceInfo(
    token="fGcm_token_123:APA91bH_example_android_fcm_token_with_proper_format",
    type=DeviceType.ANDROID,
    device_id="android_device_001",
    device_model="Samsung Galaxy S24",
    os_version="14.0",
    app_version="2.1.0",
    push_provider=PushProvider.FCM
)

android_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/Los_Angeles",
    locale="en-US",
    battery_level=0.92,
    network_type="5g",
    carrier="Verizon",
    screen_width=1440,
    screen_height=3120
)

android_registration = DeviceRegistration(
    user_id="user_mobile_002",
    device_info=android_device_info,
    attributes=android_attributes,
    status=DeviceStatus.ACTIVE
)

print("Android device registration:")
print(f"  User: {android_registration.user_id}")
print(f"  Device: {android_registration.device_info.device_model}")
print(f"  OS: Android {android_registration.device_info.os_version}")
print(f"  Carrier: {android_registration.attributes.carrier}")
print(f"  Network: {android_registration.attributes.network_type}")
print("SUCCESS: Android device registration created")

In [ ]:
# Create web browser device registration using DeviceManager models

web_device_info = DeviceInfo(
    token="BP91bH_example_web_push_token_with_base64_encoding_for_browser_notifications",
    type=DeviceType.WEB,
    device_id="web_browser_001",
    device_model="Chrome Browser",
    os_version="macOS 14.2",
    app_version="121.0.6167.85",
    push_provider=PushProvider.WEB_PUSH
)

web_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/Chicago",
    locale="en-US",
    network_type="wifi",
    screen_width=1920,
    screen_height=1080
)

web_registration = DeviceRegistration(
    user_id="user_web_001",
    device_info=web_device_info,
    attributes=web_attributes,
    status=DeviceStatus.ACTIVE
)

print("Web device registration:")
print(f"  User: {web_registration.user_id}")
print(f"  Device: {web_registration.device_info.device_model}")
print(f"  OS: {web_registration.device_info.os_version}")
print(f"  Browser Version: {web_registration.device_info.app_version}")
print(f"  Screen Size: {web_registration.attributes.screen_width}x{web_registration.attributes.screen_height}")
print("SUCCESS: Web device registration created")

In [ ]:
# Create desktop application device registration using DeviceManager models

desktop_device_info = DeviceInfo(
    token="desktop_push_token_example_for_native_desktop_application_notifications",
    type=DeviceType.DESKTOP,
    device_id="desktop_app_001",
    device_model="MacBook Pro",
    os_version="macOS 14.2",
    app_version="3.0.1",
    push_provider=PushProvider.DESKTOP
)

desktop_attributes = DeviceAttributes(
    push_enabled=True,
    timezone="America/New_York",
    locale="en-US",
    network_type="ethernet",
    screen_width=2560,
    screen_height=1600
)

desktop_registration = DeviceRegistration(
    user_id="user_desktop_001",
    device_info=desktop_device_info,
    attributes=desktop_attributes,
    status=DeviceStatus.ACTIVE
)

print("Desktop device registration:")
print(f"  User: {desktop_registration.user_id}")
print(f"  Device: {desktop_registration.device_info.device_model}")
print(f"  OS: {desktop_registration.device_info.os_version}")
print(f"  App Version: {desktop_registration.device_info.app_version}")
print(f"  Network: {desktop_registration.attributes.network_type}")
print("SUCCESS: Desktop device registration created")

## Device Lifecycle Management

In [ ]:
# Update device status using DeviceManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - status update not actually sent")
    result = {"status": "test_success", "message": "Status update validated"}
else:
    result = device_manager.update_device_status(
        user_id="user_mobile_001",
        device_token=ios_device_info.token,
        new_status=DeviceStatus.INACTIVE,
        reason="User disabled push notifications"
    )

print("Device status update using DeviceManager:")
print(f"  User: user_mobile_001")
print(f"  New Status: inactive")
print(f"  Reason: User disabled push notifications")
print(f"Result: {result}")
print("SUCCESS: Device status updated using DeviceManager")

In [ ]:
# Handle device token refresh using DeviceManager

new_ios_token = "z9y8x7w6v5u4t3s2r1q0p9o8n7m6l5k4j3i2h1g0f9e8d7c6b5a4321098765432"
token_refresh = device_manager.refresh_device_token(
    user_id="user_mobile_001",
    old_token=ios_device_info.token,
    new_token=new_ios_token,
    device_type=DeviceType.IOS
)

print("Device token refresh using DeviceManager:")
print(f"  User: user_mobile_001")
print(f"  Device Type: ios")
print(f"  Old Token: {ios_device_info.token[:20]}...")
print(f"  New Token: {new_ios_token[:20]}...")
print("SUCCESS: Device token refresh event created")

In [ ]:
# Remove device registration using DeviceManager

device_removal = device_manager.remove_device(
    user_id="user_mobile_002",
    device_token=android_device_info.token,
    device_type=DeviceType.ANDROID,
    reason="app_uninstalled"
)

print("Device removal using DeviceManager:")
print(f"  User: user_mobile_002")
print(f"  Device Type: android")
print(f"  Reason: app_uninstalled")
print(f"  Token: {android_device_info.token[:20]}...")
print("SUCCESS: Device removal event created")

## Batch Device Operations

In [ ]:
# Create batch device registrations using DeviceManager

batch_registrations = [
    ios_registration,
    android_registration,
    web_registration,
    desktop_registration
]

print(f"Created batch of {len(batch_registrations)} device registrations:")
for registration in batch_registrations:
    device_type = registration.device_info.type
    user_id = registration.user_id
    device_model = registration.device_info.device_model
    print(f"  - {user_id}: {device_type} ({device_model})")

print("SUCCESS: Batch device registrations prepared for DeviceManager")

In [ ]:
# Send batch device registrations using DeviceManager

if ENVIRONMENT == "test":
    print("INFO: Running in test mode - batch devices not actually sent")
    batch_results = [
        {"batch_id": 0, "status": "test_success", "count": len(batch_registrations)}
    ]
else:
    batch_results = device_manager.batch_register_devices(batch_registrations)

print("\nDevice batch submission results:")
for result in batch_results:
    print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} devices)")

print("SUCCESS: Batch device registrations processed using DeviceManager")

## Cross-Platform Device Tracking

In [ ]:
# Create cross-platform user using DeviceManager

# Create devices for same user for demonstration
multi_device_user_devices = [ios_registration, android_registration, web_registration]

# Update all devices to same user
for device in multi_device_user_devices:
    device.user_id = "user_cross_platform_001"

cross_platform_event = device_manager.create_cross_platform_user(
    user_id="user_cross_platform_001",
    devices=multi_device_user_devices
)

print("Cross-platform user profile using DeviceManager:")
print(json.dumps(cross_platform_event, indent=2, default=str))
print("SUCCESS: Cross-platform user profile created using DeviceManager")

In [ ]:
# Analyze device preferences using DeviceManager

all_devices = [ios_registration, android_registration, web_registration, desktop_registration]
preferences = device_manager.analyze_device_preferences(all_devices)

print("Device preference analysis using DeviceManager:")
print(json.dumps(preferences, indent=2, default=str))
print("SUCCESS: Device preferences analyzed using DeviceManager")

## Device Data from Spark Integration

In [ ]:
# Load device data from Delta table
print("=== Device Data Integration ===")

# Create sample device data table if it doesn't exist
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {CATALOG_NAME}.{DATABASE_NAME}.device_registrations (
    user_id STRING,
    device_token STRING,
    device_type STRING,
    device_model STRING,
    os_version STRING,
    app_version STRING,
    push_enabled BOOLEAN,
    timezone STRING,
    locale STRING,
    registered_at TIMESTAMP
) USING DELTA
""")

# Insert sample device data
spark.sql(f"""
INSERT INTO {CATALOG_NAME}.{DATABASE_NAME}.device_registrations
SELECT * FROM VALUES
    ('user_spark_001', 'ios_token_001', 'ios', 'iPhone 15', '17.2', '2.1.0', true, 'America/New_York', 'en-US', current_timestamp()),
    ('user_spark_001', 'android_token_001', 'android', 'Pixel 8', '14.0', '2.1.0', true, 'America/New_York', 'en-US', current_timestamp()),
    ('user_spark_002', 'web_token_001', 'web', 'Chrome', '121.0', '2.1.0', true, 'America/Los_Angeles', 'en-US', current_timestamp()),
    ('user_spark_003', 'ios_token_002', 'ios', 'iPhone 14', '17.1', '2.0.8', false, 'Europe/London', 'en-GB', current_timestamp())
WHERE NOT EXISTS (
    SELECT 1 FROM {CATALOG_NAME}.{DATABASE_NAME}.device_registrations 
    WHERE device_token = 'ios_token_001'
)
""")

# Load device registrations
devices_df = spark.table(f"{CATALOG_NAME}.{DATABASE_NAME}.device_registrations")
print("Sample device registrations from Spark:")
devices_df.show(truncate=False)

In [ ]:
# Transform Spark device data using DeviceManager models

def transform_spark_devices_to_registrations(df):
    """Transform device data from Spark to DeviceRegistration objects."""
    
    # Collect data (in production, process in batches)
    devices = df.collect()
    
    registrations = []
    for device in devices:
        # Create DeviceInfo
        device_info = DeviceInfo(
            token=device['device_token'],
            type=DeviceType(device['device_type']),
            device_model=device['device_model'],
            os_version=device['os_version'],
            app_version=device['app_version']
        )
        
        # Create DeviceAttributes
        attributes = DeviceAttributes(
            push_enabled=device['push_enabled'],
            timezone=device['timezone'],
            locale=device['locale']
        )
        
        # Create DeviceRegistration
        registration = DeviceRegistration(
            user_id=device['user_id'],
            device_info=device_info,
            attributes=attributes,
            registered_at=device['registered_at']
        )
        
        registrations.append(registration)
    
    return registrations

# Transform device data using DeviceManager models
spark_device_registrations = transform_spark_devices_to_registrations(devices_df)
print(f"Transformed {len(spark_device_registrations)} device records to DeviceRegistration objects")

# Show sample
if spark_device_registrations:
    sample = spark_device_registrations[0]
    print(f"\nSample transformed device:")
    print(f"  User: {sample.user_id}")
    print(f"  Type: {sample.device_info.type}")
    print(f"  Model: {sample.device_info.device_model}")
    print(f"  Push Enabled: {sample.attributes.push_enabled}")

In [ ]:
# Process Spark device registrations using DeviceManager

if spark_device_registrations:
    if ENVIRONMENT == "test":
        print("INFO: Running in test mode - Spark devices not actually sent")
        spark_batch_results = [
            {"batch_id": 0, "status": "test_success", "count": len(spark_device_registrations)}
        ]
    else:
        spark_batch_results = device_manager.batch_register_devices(spark_device_registrations)
    
    print("\nSpark device registrations batch results:")
    for result in spark_batch_results:
        print(f"  Batch {result['batch_id']}: {result['status']} ({result['count']} devices)")
else:
    print("No device registrations to process from Spark data")

## Device Analytics and Reporting

In [ ]:
# Calculate device metrics using DeviceManager

# Collect all device registrations for analysis
all_device_registrations = (
    [ios_registration, android_registration, web_registration, desktop_registration] + 
    spark_device_registrations
)

device_metrics = device_manager.calculate_device_metrics(all_device_registrations)

print("=== Device Analytics (using DeviceManager) ===")
print(json.dumps(device_metrics, indent=2, default=str))
print("SUCCESS: Device metrics calculated using DeviceManager")

## Performance Monitoring and Health Checks

In [ ]:
# Monitor device health using DeviceManager

health_report = device_manager.monitor_device_health(all_device_registrations)

print("=== Device Health Monitoring (using DeviceManager) ===")
print(f"Health Score: {health_report['health_score']:.2%}")
print(f"Healthy Devices: {health_report['healthy_devices']}/{health_report['total_devices']}")

if health_report["issues_found"]:
    print(f"\nIssues Found ({len(health_report['issues_found'])}):"))
    for issue in health_report["issues_found"]:
        print(f"  - {issue['user_id']} ({issue['device_type']}): {', '.join(issue['issues'])}")

if health_report["recommendations"]:
    print(f"\nRecommendations:")
    for rec in health_report["recommendations"]:
        print(f"  - {rec}")

print("SUCCESS: Device health monitoring using DeviceManager")

## Clean Up and Summary

In [ ]:
# Final summary
print("=== Device Management Summary ===")

print("\n=== Devices Registered ===")
print("SUCCESS: iOS device with APNS integration")
print("SUCCESS: Android device with FCM integration")
print("SUCCESS: Web browser with Web Push API")
print("SUCCESS: Desktop application notifications")
print("SUCCESS: Device data imported from Spark")

print("\n=== Device Operations ===")
print("SUCCESS: Device status lifecycle management")
print("SUCCESS: Push token refresh handling")
print("SUCCESS: Device removal and cleanup")
print("SUCCESS: Cross-platform user tracking")
print("SUCCESS: Batch device operations")

print("\n=== Key Capabilities Demonstrated ===")
print("SUCCESS: Type-safe device registration with validation")
print("SUCCESS: Multi-platform push notification support")
print("SUCCESS: Device lifecycle and status management")
print("SUCCESS: Cross-platform user tracking and analytics")
print("SUCCESS: Batch operations with optimization")
print("SUCCESS: Device health monitoring and diagnostics")
print("SUCCESS: Data integration from Spark DataFrames")
print("SUCCESS: Comprehensive device analytics and reporting")

In [ ]:
# Close the API client connection
client.close()
print("SUCCESS: API client connection closed")

print("\nCOMPLETED: Device management notebook finished successfully!")
print("Ready for advanced tracking operations in the next notebook.")